# [QUICKSTART](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html)

+ torch.utils.data.Dataset: 入出力データ
+ torch.utils.data.DataLoader: Datasetのイテラブル

In [1]:
import torch
import torch.nn as nn
import torchvision

torchは分野ごとに，torchtext, torchvision, torchaudioに分かれており，それらにデータセットが含まれている．

今回扱うFashionMNISTは，服の画像とそれに対応する服の種類がラベルとして与えらる．

In [2]:
train_data = torchvision.datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform = torchvision.transforms.ToTensor()
)
test_data = torchvision.datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform = torchvision.transforms.ToTensor()
)

In [3]:
print('type: {}'.format(type(train_data)))
print('content: \n{}'.format(train_data))


type: <class 'torchvision.datasets.mnist.FashionMNIST'>
content: 
Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()


In [4]:
batch_size = 32
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    # [batch_size, 1, 28, 28]
    # 白黒画像の28x28px
    print('X shape: {}'.format(X.shape))
    # [batch_size]
    # 0~9のラベル
    print('y shape: {}'.format(y.shape))
    break

X shape: torch.Size([32, 1, 28, 28])
y shape: torch.Size([32])


In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"loss: {loss:.4f}, [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.2987, [    0/60000]
loss: 2.2949, [ 3200/60000]
loss: 2.2759, [ 6400/60000]
loss: 2.2575, [ 9600/60000]
loss: 2.2462, [12800/60000]
loss: 2.2354, [16000/60000]
loss: 2.2124, [19200/60000]
loss: 2.1947, [22400/60000]
loss: 2.2016, [25600/60000]
loss: 2.1492, [28800/60000]
loss: 2.1119, [32000/60000]
loss: 2.0823, [35200/60000]
loss: 2.1734, [38400/60000]
loss: 2.0745, [41600/60000]
loss: 2.0406, [44800/60000]
loss: 2.0158, [48000/60000]
loss: 2.0871, [51200/60000]
loss: 1.9880, [54400/60000]
loss: 1.9264, [57600/60000]
Test Error: 
 Accuracy: 58.9%, Avg loss: 1.913935 

Epoch 2
-------------------------------
loss: 1.8986, [    0/60000]
loss: 1.9435, [ 3200/60000]
loss: 1.8585, [ 6400/60000]
loss: 1.8010, [ 9600/60000]
loss: 1.7402, [12800/60000]
loss: 1.7786, [16000/60000]
loss: 1.6198, [19200/60000]
loss: 1.6079, [22400/60000]
loss: 1.6080, [25600/60000]
loss: 1.5703, [28800/60000]
loss: 1.4109, [32000/60000]
loss: 1.3394, [35200/60000]


In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
